In [ ]:
from gurobipy import *
import pandas as pd

# DONNÉES DU PROBLÈME
df_weights = pd.read_excel(
    "RATP.xlsx",
    usecols="C:I",
    skiprows=13,
    nrows=1,
) # type: ignore

# Notes des candidats u et v
data = pd.read_excel(
    "RATP.xlsx",
    usecols="B:I",
    skiprows=2,
    nrows=10,
    index_col=0,
)
df_weights.columns = data.columns
weights = df_weights.iloc[0].to_dict()

# Pour chaque paire (u,v) on effectue l'analyse précédente

for u in data.index:
    for v in data.index:
        if u == v:
            continue
        
        print(f"\n\n--- Analyse de l'explication entre {u} et {v} ---")

        # Calcul des contributions (deltas)
        deltas = {}
        for k in weights:
            deltas[k] = weights[k] * (data.loc[u, k] - data.loc[v, k])  

        pros = [k for k, v in deltas.items() if v > 0]
        cons = [k for k, v in deltas.items() if v < 0]

        print(f"Arguments Pour (Pros) : {pros}")
        print(f"Arguments Contre (Cons) : {cons}")

        # MODÉLISATION GUROBI
        m = Model("Explication_1_m")

        # Variables de décision UNIQUEMENT x
        # x[p, c] = 1 si l'argument Pro 'p' couvre l'argument Con 'c'
        x = m.addVars(pros, cons, vtype=GRB.BINARY, name="x")

        m.update()

        # CONTRAINTES

        # A. Couverture complète : Chaque Con doit être couvert par exactement un Pro
        for c in cons:
            m.addConstr(quicksum(x[p, c] for p in pros) == 1, name=f"Cover_{c}")

        # B. Validité du Trade-off
        # Formule : Delta_p + Somme(Delta_c * x_pc) >= epsilon
        # Si p est utilisé, cela vérifie que le trade-off est positif.
        # Si p est INUTILISÉ (x_pc = 0 partout), cela devient Delta_p >= epsilon, ce qui est toujours VRAI.
        epsilon = 0.001 

        for p in pros:
            # Note: deltas[c] est négatif ici
            m.addConstr(deltas[p] + quicksum(deltas[c] * x[p, c] for c in cons) >= epsilon, name=f"Valid_{p}")

        # RÉSOLUTION
        m.params.outputflag = 0  
        m.optimize()

        # ANALYSE
        print("-" * 30)
        if m.status == GRB.OPTIMAL:
            print("Résultat : Une explication de type (1-m) existe !\n")
            
            # On itère sur les Pros et on vérifie s'ils ont des Cons assignés
            for p in pros:
                covered_cons = [c for c in cons if x[p, c].x > 0.5]
                
                # Si la liste n'est pas vide, c'est que ce Pro fait partie de l'explication
                if covered_cons:
                    net_value = deltas[p] + sum(deltas[c] for c in covered_cons)
                    print(f"Groupe mené par {p} (+{deltas[p]}) :")
                    print(f"  -> Couvre : {covered_cons} (Poids cumulé Cons: {sum(deltas[c] for c in covered_cons)})")
                    print(f"  -> Bilan net : +{net_value}")

        elif m.status == GRB.INFEASIBLE:
            print("Résultat : INFEASIBLE (Certificat de non-existence).")